In [204]:
import re
from modulefinder import replacePackageMap


class Token:
    def __init__(self, name, regex):
        self.name = name
        self.regex = re.compile(regex)

    def matches(self, text):
        return self.regex.match(text) is not None

    def __repr__(self):
        return str(self.name)

    def __str__(self):
        return str(self.name)

In [205]:

class BinaryOperator(Token):
    def __init__(self, name, regex, precedence, fn=None):
        super().__init__(name, regex)
        self.precedence = precedence
        self.fn = fn

    def __repr__(self):
        return f'<{self.name}, precedence={self.precedence}>'

    def __gt__(self, other):
        return self.precedence > other.precedence

    def __lt__(self, other):
        return self.precedence < other.precedence

    def __eq__(self, other):
        return self.precedence == other.precedence

In [206]:
plus = BinaryOperator('+', r'\+', 1, lambda a, b: a + b)
minus = BinaryOperator('-', r'\-', 1, lambda a, b: a - b)
times = BinaryOperator('*', r'\*', 2, lambda a, b: a * b)
divide = BinaryOperator('/', r'\/', 2, lambda a, b: a / b)
operators = [
    plus,
    minus,
    times,
    divide,
]

In [207]:
class Bracket(Token):
    def __init__(self, name, regex, pair=None):
        super().__init__(name, regex)
        self.pair = pair

    def __repr__(self):
        return f'<bracket {self.name}, matches {self.pair}>'

    def set_pair(self, other):
        self.pair = other

    def pairs_with(self, other):
        return self.pair == other


class LeftBracket(Bracket):
    def __init__(self, name, regex, pair=None):
        super().__init__(name, regex, pair)

    def __repr__(self):
        return f'<left bracket {self.name}, matches {self.pair}>'

class RightBracket(Bracket):
    def __init__(self, name, regex, pair=None):
        super().__init__(name, regex, pair)

    def __repr__(self):
        return f'<right bracket {self.name}, matches {self.pair}>'


In [208]:
bracket_left = LeftBracket('(', r'\(')
bracket_right = RightBracket(')', r'\)', pair=bracket_left)

bracket_left.set_pair(bracket_right)

In [209]:
brackets = [
    bracket_left,
    bracket_right,
]

In [232]:
class Operand(Token):
    def __init__(self, name, regex, value=None):
        super().__init__(name, regex)
        self.value = value

    def __str__(self):
        return f'{self.value}'

    def __repr__(self):
        return f'<{self.name}, value={self.value}>'

class Number(Operand):
    regex = re.compile(r'\d+')
    def __init__(self, value):
        super().__init__(name='number', regex=r'\d+', value=value)

In [222]:
class InFix(list):
    def __str__(self):
        return ' '.join(map(str, self))

In [223]:
infix_tokens = InFix([
    bracket_left,
    bracket_left,
    Number(12),
    plus,
    Number(10),
    bracket_right,
    times,
    Number(3),
    bracket_right,
    divide,
    bracket_left,
    Number(13),
    minus,
    Number(2),
    bracket_right,
    ]
)

In [224]:
infix_tokens

[<left bracket (, matches )>,
 <left bracket (, matches )>,
 <number, value=12>,
 <+, precedence=1>,
 <number, value=10>,
 <right bracket ), matches (>,
 <*, precedence=2>,
 <number, value=3>,
 <right bracket ), matches (>,
 </, precedence=2>,
 <left bracket (, matches )>,
 <number, value=13>,
 <-, precedence=1>,
 <number, value=2>,
 <right bracket ), matches (>]

In [225]:
print(infix_tokens)

( ( 12 + 10 ) * 3 ) / ( 13 - 2 )


In [226]:
class RPN(list):
    def __str__(self):
        return ' '.join(map(str, self))

    def eval(self, log=False):
        rpn_stack = []

        for t in self:
            if isinstance(t, BinaryOperator):
                b = rpn_stack.pop()
                a = rpn_stack.pop()
                c = t.fn(a.value, b.value)
                rpn_stack.append(Number(c))
            else:
                rpn_stack.append(t)

            if log:
                print(f'Processed {t} giving {rpn_stack}')
        return rpn_stack.pop()


In [234]:
def infix_to_rpn(infix_tokens, log=False):
    siding_stack = []
    rpn_output = RPN()

    # Work through the infix tokens

    for token in infix_tokens:

        # Numbers go straight to our RPN output

        if isinstance(token, Number):
            rpn_output.append(token)

        # Left brackets go straight onto the siding

        elif isinstance(token, LeftBracket):
            siding_stack.append(token)

        # Right brackets mean that the calculation thus far is higher
        # precedence than whatever follows, and so we push operators
        # to the output until we reach a left bracket

        elif isinstance(token, RightBracket):

            popping_operators = True

            while popping_operators:
                next_token = siding_stack.pop()
                if isinstance(next_token, BinaryOperator):
                    rpn_output.append(next_token)
                elif isinstance(next_token, LeftBracket):
                    popping_operators = False

        elif isinstance(token, BinaryOperator):
            # If we have an operator then we need to look to see what's on the
            # siding stack. We start by checking that it isn't empty.

            if siding_stack:
                # If there's a bracket then... no precedence will do...
                # Nothing to see here, move on

                if isinstance(siding_stack[-1], LeftBracket):
                    pass

                # Otherwise we pop all operators with higher precedence
                # There may be no popping at all, notice
                elif isinstance(siding_stack[-1], BinaryOperator):
                    while siding_stack[-1] >= token:
                        rpn_output.append(siding_stack.pop())

            # And finally our operator can go on the siding stack
            siding_stack.append(token)

        if log:
            print(f'Processed token: {token}')
            print(f'Output: {rpn_output}')
            print(f'Siding stack: {siding_stack}')
            print('')

    while siding_stack:
        rpn_output.append(siding_stack.pop())

        if log:
            print(f'Output: {rpn_output}')
            print(f'Siding stack: {siding_stack}')
            print('')
    return rpn_output

In [235]:
rpn = infix_to_rpn(infix_tokens, log=True)


Processed token: (
Output: 
Siding stack: [<left bracket (, matches )>]

Processed token: (
Output: 
Siding stack: [<left bracket (, matches )>, <left bracket (, matches )>]

Processed token: 12
Output: 
Siding stack: [<left bracket (, matches )>, <left bracket (, matches )>]

Processed token: +
Output: 
Siding stack: [<left bracket (, matches )>, <left bracket (, matches )>, <+, precedence=1>]

Processed token: 10
Output: 
Siding stack: [<left bracket (, matches )>, <left bracket (, matches )>, <+, precedence=1>]

Processed token: )
Output: +
Siding stack: [<left bracket (, matches )>]

Processed token: *
Output: +
Siding stack: [<left bracket (, matches )>, <*, precedence=2>]

Processed token: 3
Output: +
Siding stack: [<left bracket (, matches )>, <*, precedence=2>]

Processed token: )
Output: + *
Siding stack: []

Processed token: /
Output: + *
Siding stack: [</, precedence=2>]

Processed token: (
Output: + *
Siding stack: [</, precedence=2>, <left bracket (, matches )>]

Processed

In [236]:
rpn.eval(log=True)

IndexError: pop from empty list

In [264]:
language = [
    Number,
    plus,
    minus,
    times,
    divide,
    bracket_left,
    bracket_right,
]
def tokenize_infix(infix_text, log=False):
    tokens = InFix()
    processing = True
    while processing:
        if log:
            print(f'Processing {infix_text}')
        processing = False
        for t in language:
            m = t.regex.match(infix_text)
            if m:
                if log:
                    print(f'Matched {m.group()} as {t}')
                processing = True
                infix_text = infix_text[m.end():]
                if t == Number:
                    tokens.append(Number(m.group()))
                else:
                    tokens.append(t)
                break
        print(f'InFix tokens: {tokens}')
    return tokens

In [265]:
tokenize_infix('3*2-7', True)

Processing 3*2-7
Matched 3 as <class '__main__.Number'>
InFix tokens: 3
Processing *2-7
Matched * as *


AttributeError: type object 'Number' has no attribute 'precedence'